0    Wayona Nylon Braided USB to Lightning Fast Cha...
1    Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2    Sounce Fast Phone Charging Cable & Data Sync U...
3    boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4    Portronics Konnect L 1.2M Fast Charging 3A 8 P...
Name: product_name, dtype: object
Produto 'wayona nylon braided usb to lightning fast charger' não encontrado no dataset.


In [3]:
# Verificar valores nulos
df.isnull().sum()

# Preencher valores nulos se necessário (exemplo)
df['about_product'] = df['about_product'].fillna('')


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Criar a matriz TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['about_product'])

# Calcular a similaridade entre produtos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Carregar o dataset
df = pd.read_csv('data/amazon.csv')

# Normalizar o nome do produto
def normalize_text(text):
    return text.lower()

# Criar a matriz TF-IDF para as descrições dos produtos
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['about_product'])

# Calcular a similaridade entre produtos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(product_name, cosine_sim=cosine_sim):
    # Normalizar o nome do produto
    product_name = normalize_text(product_name)

    # Encontrar o índice do produto com base no nome
    idx_list = df.index[df['product_name'].str.lower() == product_name].tolist()
    
    if not idx_list:
        print(f"Produto '{product_name}' não encontrado no dataset.")
        return None
    
    idx = idx_list[0]
    
    # Obter a similaridade dos produtos
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar produtos com base na similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obter os índices dos produtos recomendados
    sim_scores = sim_scores[1:6]  # Excluir o próprio produto
    product_indices = [i[0] for i in sim_scores]
    
    # Retornar os produtos recomendados
    return df.iloc[product_indices]

# Testar com um dos nomes de produtos fornecidos
product_names_to_test = [
    'Wayona Nylon Braided USB to Lightning Fast Charger',
    'Ambrane Unbreakable 60W / 3A Fast Charging 1.5 Meter Type C Cable',
    'Sounce Fast Phone Charging Cable & Data Sync USB Charger',
    'boAt Deuce USB 300 2 in 1 Type-C & Micro USB Sync Cable',
    'Portronics Konnect L 1.2M Fast Charging 3A 8 Pin Data Cable'
]

for product_name in product_names_to_test:
    print(f"Testando produto: {product_name}")
    recommendations = get_recommendations(product_name)
    if recommendations is not None:
        print(recommendations[['product_name', 'discounted_price', 'rating']])
    print()


Testando produto: Wayona Nylon Braided USB to Lightning Fast Charger
Produto 'wayona nylon braided usb to lightning fast charger' não encontrado no dataset.

Testando produto: Ambrane Unbreakable 60W / 3A Fast Charging 1.5 Meter Type C Cable
Produto 'ambrane unbreakable 60w / 3a fast charging 1.5 meter type c cable' não encontrado no dataset.

Testando produto: Sounce Fast Phone Charging Cable & Data Sync USB Charger
Produto 'sounce fast phone charging cable & data sync usb charger' não encontrado no dataset.

Testando produto: boAt Deuce USB 300 2 in 1 Type-C & Micro USB Sync Cable
Produto 'boat deuce usb 300 2 in 1 type-c & micro usb sync cable' não encontrado no dataset.

Testando produto: Portronics Konnect L 1.2M Fast Charging 3A 8 Pin Data Cable
Produto 'portronics konnect l 1.2m fast charging 3a 8 pin data cable' não encontrado no dataset.



In [30]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

# Dicionário de mapeamento
term_mapping = {
    'celular': 'phone',
    'smartphone': 'phone',
    'computador': 'computer',
    'notebook': 'laptop',
    'acessórios': 'accessories',
    'cabo': 'cable',
    'carregador': 'charger',
    'ventilador': 'fan',
    'cozinha': 'kitchen',
    'fritadeira': 'fryer',
    'televisão': 'tv',
    'televisor': 'tv',
    'refrigerador': 'fridge',
    'geladeira': 'fridge',
    'micro-ondas': 'microwave',
    'forno': 'oven',
    'cafeteira': 'coffee maker',
    'liquidificador': 'blender',
    'torradeira': 'toaster',
    'máquina de lavar': 'washing machine',
    'secadora': 'dryer',
    'ar condicionado': 'air conditioner',
    'ventilador de teto': 'ceiling fan',
    'aquecedor': 'heater',
    'aspirador': 'vacuum cleaner',
    'máquina de café': 'coffee machine',
    'panelas': 'pots',
    'frigideira': 'frying pan',
    'tábua de corte': 'cutting board',
    'processador de alimentos': 'food processor',
    'escova de cabelo': 'hairbrush',
    'secador de cabelo': 'hair dryer',
    'chapinha': 'hair straightener',
    'epilador': 'epilator',
    'relógio': 'watch',
    'pulseira': 'bracelet',
    'brinco': 'earring',
    'colar': 'necklace',
    'anel': 'ring',
    'óculos': 'glasses',
    'lupa': 'magnifying glass',
    'câmera': 'camera',
    'tripé': 'tripod',
    'microfone': 'microphone',
    'alto-falante': 'speaker',
    'fone de ouvido': 'headphones',
    'mouse': 'mouse',
    'teclado': 'keyboard',
    'impressora': 'printer',
    'scanner': 'scanner',
    'modem': 'modem',
    'roteador': 'router'
}

# Função para normalizar texto
def normalize_text(text):
    return text.lower().strip()

# Função para mapear termos
def map_terms(text, term_mapping):
    for pt_term, en_term in term_mapping.items():
        text = text.replace(pt_term, en_term)
    return text

# Carregar o dataset
df = pd.read_csv('data/amazon.csv')

# Tratar dados faltantes
df.fillna({
    'product_name': 'Unknown',
    'about_product': 'No description available',
    'discounted_price': '0',
    'actual_price': '0',
    'rating': '0',
    'rating_count': '0'
}, inplace=True)

# Normalizar e mapear nomes de produtos
df['normalized_product_name'] = df['product_name'].apply(lambda x: map_terms(normalize_text(x), term_mapping))

# Criar a matriz TF-IDF para as descrições dos produtos
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['about_product'])

# Calcular a similaridade entre produtos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Função de recomendação
def get_recommendations(product_name, cosine_sim=cosine_sim):
    # Normalizar e mapear o nome do produto
    product_name = map_terms(normalize_text(product_name), term_mapping)
    
    # Buscar produtos que contêm o nome fornecido
    matching_products = df[df['normalized_product_name'].str.contains(product_name, na=False)]
    
    if matching_products.empty:
        print(f"Não temos o produto '{product_name}' em nosso estoque para te recomendar.")
        return None
    
    # Usar o primeiro produto encontrado
    idx = matching_products.index[0]
    
    # Obter a similaridade dos produtos
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar produtos com base na similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obter os índices dos produtos recomendados
    sim_scores = sim_scores[1:6]  # Excluir o próprio produto
    product_indices = [i[0] for i in sim_scores]
    
    # Retornar os produtos recomendados
    return df.iloc[product_indices]

# Testar a função com uma pesquisa em português
test_queries = ['celular', 'carregador', 'ventilador', 'cozinha', 'televisor', 'liquidificador', 'secador de cabelo']

for query in test_queries:
    print(f"Recomendações para '{query}':")
    recommendations = get_recommendations(query)
    if recommendations is not None:
        print(recommendations[['product_name', 'discounted_price', 'rating']])
    print()

# Salvar o modelo TF-IDF e a matriz de similaridade



Recomendações para 'celular':
                                          product_name discounted_price rating
369  Wayona Nylon Braided USB to Lightning Fast Cha...             ₹399    4.2
614  Wayona Nylon Braided USB to Lightning Fast Cha...             ₹399    4.2
220  Wayona Nylon Braided Usb Syncing And Charging ...             ₹649    4.2
42   Wayona Nylon Braided 3A Lightning to USB A Syn...             ₹399    4.2
89   Wayona Nylon Braided (2 Pack) Lightning Fast U...             ₹649    4.2

Recomendações para 'carregador':
                                          product_name discounted_price rating
823  Zoul USB C 60W Fast Charging 3A 6ft/2M Long Ty...             ₹389    4.3
151  Zoul USB Type C Fast Charging 3A Nylon Braided...             ₹339    4.3
45   Wecool Nylon Braided Multifunction Fast Chargi...             ₹333    3.3
857  Wecool Nylon Braided Multifunction Fast Chargi...             ₹333    3.3
237  Wayona 3in1 Nylon Braided 66W USB Fast Chargin...             

In [31]:
with open('tfidf_model.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
with open('cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)